In [1]:
from dataloader import *
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2 as cv # so annoying in tutorials ngl

In [2]:
files = DataAnuncis('/home/adri/Desktop/cvc/data/tinder-historic/filenames.txt')


In [61]:
sample = files[2]

In [62]:
plt.imshow(sample, cmap = 'gray')
plt.show()

In [63]:
ret, thresh = cv2.threshold(sample, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)


In [64]:
plt.imshow(thresh, cmap = 'gray')
plt.show()

In [65]:
cols = thresh.shape[1]
horizontal_size = cols // 30
horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size, 1))
horizontal = cv2.erode(thresh, horizontalStructure)
horizontal = cv2.dilate(horizontal, horizontalStructure)

In [66]:
# Specify size on vertical axis
rows = thresh.shape[0]
verticalsize = rows // 30
# Create structure element for extracting vertical lines through morphology operations
verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, verticalsize))
# Apply morphology operations
vertical = cv2.erode(thresh, verticalStructure)
vertical = cv2.dilate(vertical, verticalStructure)

In [67]:
%matplotlib qt
plt.imshow(vertical*.5 + horizontal*.5, cmap = 'gray')
plt.show()

In [68]:
anuncis = vertical*.5 + horizontal*.5 # What will we do when the blocks arent delimited by lines
anuncis = ((anuncis - anuncis.min()) / (anuncis.max() - anuncis.min())) * 255
ret, anuncis = cv2.threshold(anuncis.astype(np.uint8), 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
plt.imshow(anuncis, cmap = 'gray')
plt.show()

In [91]:
anuncis = ~ anuncis
plt.imshow(anuncis, cmap = 'gray')
plt.axis('off')
plt.show()


In [70]:
lines = cv.HoughLinesP(anuncis, 1, np.pi / 180, 150, 10, None)

In [71]:
src = np.zeros_like(sample)
for i in range(0, len(lines)):
    l = lines[i][0]
    src = cv.line(src, (l[0], l[1]), (l[2], l[3]), (155,155,255), 3, cv.LINE_AA)

In [72]:
plt.imshow(src, cmap = 'gray')
plt.axis('off')
plt.show()

In [73]:
contours, hierarchy = cv2.findContours(anuncis, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
print(len(contours))

398


In [87]:
sample2 = np.zeros_like(sample.copy())
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    # Drawing a rectangle on copied image
    sample2 = cv2.rectangle(sample2, (x, y), (x + w, y + h), (255, 255, 255), 2)

In [89]:
plt.imshow(sample2*0.8 + sample*0.2, cmap = 'gray')
plt.show()

In [97]:
dilation = cv2.dilate(anuncis,np.ones((3, 3)),iterations = 10)

In [99]:
plt.imshow(~dilation, cmap = 'gray')
plt.show()

In [100]:
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
print(len(contours))

99


In [101]:
sample2 = np.zeros_like(sample.copy())
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    # Drawing a rectangle on copied image
    sample2 = cv2.rectangle(sample2, (x, y), (x + w, y + h), (255, 255, 255), 2)

In [103]:
plt.imshow(sample2*0.8 + sample*0.2, cmap = 'gray')
plt.axis('off')
plt.show()